In [ ]:
import heapq
import numpy as np
import matplotlib.pyplot as plt
def manhattan(p1, p2):
    return abs(p1[0] - p2[0]) + abs(p1[1] - p2[1])

def get_neighbors(pos, maze):
    directions = [(0,1),(1,0),(0,-1),(-1,0)]
    neighbors = []
    for dx, dy in directions:
        nx, ny = pos[0]+dx, pos[1]+dy
        if 0 <= nx < maze.shape[0] and 0 <= ny < maze.shape[1]:
            if maze[nx][ny] != 1:  # not a wall
                neighbors.append((nx, ny))
    return neighbors

def reconstruct_path(came_from, current):
    path = []
    while current in came_from:
        path.append(current)
        current = came_from[current]
    path.reverse()
    return path


In [2]:
def astar(maze, start, goal, heuristic):
    open_set = []
    heapq.heappush(open_set, (0, start))
    
    came_from = {}
    g_score = {start: 0}
    f_score = {start: heuristic(start, goal)}
    
    while open_set:
        _, current = heapq.heappop(open_set)
        
        if current == goal:
            path = reconstruct_path(came_from, current)
            return path, g_score[current]
        
        for neighbor in get_neighbors(current, maze):
            tentative_g = g_score[current] + 1
            if tentative_g < g_score.get(neighbor, float('inf')):
                came_from[neighbor] = current
                g_score[neighbor] = tentative_g
                f_score[neighbor] = tentative_g + heuristic(neighbor, goal)
                heapq.heappush(open_set, (f_score[neighbor], neighbor))
    
    return None, float('inf')


In [3]:
def visualize_maze(maze, path, start, goal):
    maze_copy = maze.copy().astype(str)
    maze_copy[maze_copy=='1'] = '#'
    maze_copy[maze_copy=='0'] = '.'
    
    for (x,y) in path:
        if (x,y) != start and (x,y) != goal:
            maze_copy[x,y] = '*'
    
    maze_copy[start] = 'A'
    maze_copy[goal] = 'B'
    
    for row in maze_copy:
        print(" ".join(row))


In [4]:
maze = np.array([
    ['A','0','1','0','0'],
    ['1','0','1','0','1'],
    ['0','0','0','0','0'],
    ['0','1','1','1','0'],
    ['0','0','0','B','0']
])

start = tuple(np.argwhere(maze=='A')[0])
goal = tuple(np.argwhere(maze=='B')[0])

path, cost = astar(maze, start, goal, manhattan)
print("Path:", path)
print("Cost:", cost)
visualize_maze(maze, path, start, goal)


Path: [(np.int64(0), np.int64(1)), (np.int64(0), np.int64(2)), (np.int64(0), np.int64(3)), (np.int64(1), np.int64(3)), (np.int64(2), np.int64(3)), (np.int64(3), np.int64(3)), (np.int64(4), np.int64(3))]
Cost: 7
A * * * .
# . # * #
. . . * .
. # # * .
. . . B .


In [5]:
def heuristic_1_5(p1, p2):
    return 1.5 * manhattan(p1, p2)


In [6]:
def inconsistent_heuristic(p1, p2):
    h = manhattan(p1, p2)
    if p1[0] == 2 and p1[1] == 2:  # special edge
        return h + 5   # violates consistency
    return h


In [7]:
for h, name in [(manhattan,"Manhattan"), 
                (heuristic_1_5,"1.5 * Manhattan"), 
                (inconsistent_heuristic,"Inconsistent")]:
    
    print("\nHeuristic:", name)
    path, cost = astar(maze, start, goal, h)
    print("Path:", path)
    print("Cost:", cost)
    if path:
        visualize_maze(maze, path, start, goal)



Heuristic: Manhattan
Path: [(np.int64(0), np.int64(1)), (np.int64(0), np.int64(2)), (np.int64(0), np.int64(3)), (np.int64(1), np.int64(3)), (np.int64(2), np.int64(3)), (np.int64(3), np.int64(3)), (np.int64(4), np.int64(3))]
Cost: 7
A * * * .
# . # * #
. . . * .
. # # * .
. . . B .

Heuristic: 1.5 * Manhattan
Path: [(np.int64(0), np.int64(1)), (np.int64(0), np.int64(2)), (np.int64(0), np.int64(3)), (np.int64(1), np.int64(3)), (np.int64(2), np.int64(3)), (np.int64(3), np.int64(3)), (np.int64(4), np.int64(3))]
Cost: 7
A * * * .
# . # * #
. . . * .
. # # * .
. . . B .

Heuristic: Inconsistent
Path: [(np.int64(0), np.int64(1)), (np.int64(0), np.int64(2)), (np.int64(0), np.int64(3)), (np.int64(1), np.int64(3)), (np.int64(2), np.int64(3)), (np.int64(3), np.int64(3)), (np.int64(4), np.int64(3))]
Cost: 7
A * * * .
# . # * #
. . . * .
. # # * .
. . . B .


In [1]:
import heapq
import numpy as np
import matplotlib.pyplot as plt

def manhattan_distance(pos, goal):
    return abs(pos[0] - goal[0]) + abs(pos[1] - goal[1])

def heuristic_1_5_manhattan(pos, goal):
    return 1.5 * manhattan_distance(pos, goal)

def inconsistent_heuristic(pos, goal):
    base_manhattan = manhattan_distance(pos, goal)
    if pos == (2, 3):
        return base_manhattan + 5
    return base_manhattan

def astar_search(maze, start, goal, heuristic_func):
    rows, cols = len(maze), len(maze[0])
    
    open_set = []
    heapq.heappush(open_set, (0, start))
    
    came_from = {}
    g_score = {start: 0}
    f_score = {start: heuristic_func(start, goal)}
    closed_set = set()
    nodes_expanded = 0
    
    while open_set:
        current_f, current = heapq.heappop(open_set)
        
        if current in closed_set:
            continue
            
        closed_set.add(current)
        nodes_expanded += 1
        
        if current == goal:
            path = reconstruct_path(came_from, current, start)
            return path, g_score[goal], nodes_expanded
        
        neighbors = get_neighbors(current, maze)
        for neighbor in neighbors:
            if neighbor in closed_set:
                continue
                
            tentative_g_score = g_score[current] + 1
            
            if neighbor not in g_score or tentative_g_score < g_score[neighbor]:
                came_from[neighbor] = current
                g_score[neighbor] = tentative_g_score
                f_score[neighbor] = tentative_g_score + heuristic_func(neighbor, goal)
                heapq.heappush(open_set, (f_score[neighbor], neighbor))
    
    return [], float('inf'), nodes_expanded

def get_neighbors(pos, maze):
    rows, cols = len(maze), len(maze[0])
    neighbors = []
    
    for dx, dy in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
        new_pos = (pos[0] + dx, pos[1] + dy)
        
        if (0 <= new_pos[0] < rows and 
            0 <= new_pos[1] < cols and 
            maze[new_pos[0]][new_pos[1]] != 1):
            neighbors.append(new_pos)
    
    return neighbors

def reconstruct_path(came_from, current, start):
    path = [current]
    while current in came_from:
        current = came_from[current]
        path.append(current)
    path.reverse()
    return path

def find_maze_positions(maze):
    start, goal = None, None
    for i in range(len(maze)):
        for j in range(len(maze[0])):
            if maze[i][j] == 'A':
                start = (i, j)
            elif maze[i][j] == 'B':
                goal = (i, j)
    return start, goal

def create_numeric_maze(maze):
    numeric_maze = []
    for row in maze:
        numeric_row = []
        for cell in row:
            if cell == 1 or cell == '1':
                numeric_row.append(1)
            else:
                numeric_row.append(0)
        numeric_maze.append(numeric_row)
    return numeric_maze

def visualize_maze_text(maze, path, start, goal, title="Maze Solution"):
    print(f"\n{title}")
    print("=" * len(title))
    
    rows, cols = len(maze), len(maze[0])
    display = [['.' if maze[i][j] == 0 else '#' for j in range(cols)] for i in range(rows)]
    
    for pos in path:
        if pos != start and pos != goal:
            display[pos[0]][pos[1]] = '*'
    
    if start:
        display[start[0]][start[1]] = 'S'
    if goal:
        display[goal[0]][goal[1]] = 'G'
    
    for row in display:
        print(' '.join(row))
    print()

def visualize_maze_plot(maze, path, start, goal, title="Maze Solution"):
    maze_array = np.array(maze)
    fig, ax = plt.subplots(1, 1, figsize=(10, 8))
    
    ax.imshow(maze_array, cmap='binary', origin='upper')
    
    if path:
        path_x = [pos[1] for pos in path]
        path_y = [pos[0] for pos in path]
        ax.plot(path_x, path_y, 'r-', linewidth=3, label='Path')
        ax.plot(path_x, path_y, 'ro', markersize=4)
    
    if start:
        ax.plot(start[1], start[0], 'go', markersize=15, label='Start')
    if goal:
        ax.plot(goal[1], goal[0], 'bo', markersize=15, label='Goal')
    
    ax.set_title(title)
    ax.legend()
    ax.grid(True, alpha=0.3)
    plt.show()

def check_admissibility(maze, start, goal, heuristic_func):
    optimal_path, optimal_cost, _ = astar_search(maze, start, goal, manhattan_distance)
    
    print(f"Optimal cost (using Manhattan): {optimal_cost}")
    
    max_overestimate = 0
    total_positions = 0
    overestimate_count = 0
    
    for i in range(len(maze)):
        for j in range(len(maze[0])):
            if maze[i][j] == 0:
                pos = (i, j)
                heuristic_value = heuristic_func(pos, goal)
                
                actual_path, actual_cost, _ = astar_search(maze, pos, goal, manhattan_distance)
                if actual_cost != float('inf'):
                    total_positions += 1
                    if heuristic_value > actual_cost:
                        overestimate_count += 1
                        overestimate = heuristic_value - actual_cost
                        max_overestimate = max(max_overestimate, overestimate)
    
    is_admissible = overestimate_count == 0
    
    print(f"Admissibility Check:")
    print(f"  Positions checked: {total_positions}")
    print(f"  Positions with overestimate: {overestimate_count}")
    print(f"  Maximum overestimate: {max_overestimate:.2f}")
    print(f"  Is Admissible: {is_admissible}")
    
    return is_admissible

def check_consistency(maze, heuristic_func):
    violations = []
    
    for i in range(len(maze)):
        for j in range(len(maze[0])):
            if maze[i][j] == 0:
                pos = (i, j)
                neighbors = get_neighbors(pos, maze)
                
                for neighbor in neighbors:
                    h_current = heuristic_func(pos, (len(maze)-1, len(maze[0])-1))
                    h_neighbor = heuristic_func(neighbor, (len(maze)-1, len(maze[0])-1))
                    step_cost = 1
                    
                    if h_current > step_cost + h_neighbor:
                        violation = h_current - (step_cost + h_neighbor)
                        violations.append((pos, neighbor, violation))
    
    is_consistent = len(violations) == 0
    
    print(f"Consistency Check:")
    print(f"  Total violations: {len(violations)}")
    if violations:
        print(f"  Sample violations:")
        for i, (pos, neighbor, violation) in enumerate(violations[:3]):
            print(f"    {pos} -> {neighbor}: violation = {violation:.2f}")
    print(f"  Is Consistent: {is_consistent}")
    
    return is_consistent, violations

def run_experiment():
    maze = [
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 1, 1, 0, 1, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 0, 1, 0],
        [0, 1, 1, 1, 1, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 1, 0, 1, 0, 0, 0],
        [1, 1, 0, 0, 0, 0, 0, 0, 1, 0],
        [0, 0, 0, 1, 1, 1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 1, 1, 0],
        [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
    ]
    
    start = (0, 0)  # A
    goal = (9, 9)   # B
    
    print("PART C: A* SEARCH IMPLEMENTATION AND ANALYSIS")
    print("=" * 50)
    
    heuristics = [
        (manhattan_distance, "Manhattan Distance"),
        (heuristic_1_5_manhattan, "1.5 * Manhattan Distance"),
        (inconsistent_heuristic, "Inconsistent Heuristic")
    ]
    
    results = []
    
    for heuristic_func, name in heuristics:
        print(f"\n{'='*60}")
        print(f"TESTING: {name}")
        print(f"{'='*60}")
        
        path, cost, nodes_expanded = astar_search(maze, start, goal, heuristic_func)
        
        print(f"Results:")
        print(f"  Path found: {len(path) > 0}")
        print(f"  Path: {path}")
        print(f"  Cost: {cost}")
        print(f"  Nodes expanded: {nodes_expanded}")
        
        if name == "Manhattan Distance":
            optimal_cost = cost
            print(f"  Optimal: True (baseline)")
        else:
            is_optimal = cost == optimal_cost if cost != float('inf') else False
            print(f"  Optimal: {is_optimal}")
        
        visualize_maze_text(maze, path, start, goal, f"Solution using {name}")
        
        if "1.5" in name:
            print(f"\nADMISSIBILITY ANALYSIS:")
            check_admissibility(maze, start, goal, heuristic_func)
        
        if "Inconsistent" in name:
            print(f"\nCONSISTENCY ANALYSIS:")
            check_consistency(maze, heuristic_func)
        
        results.append({
            'name': name,
            'path': path,
            'cost': cost,
            'nodes_expanded': nodes_expanded,
            'optimal': cost == optimal_cost if 'optimal_cost' in locals() else True
        })
    
    print(f"\n{'='*60}")
    print("SUMMARY OF RESULTS")
    print(f"{'='*60}")
    print(f"{'Heuristic':<25} {'Cost':<8} {'Optimal':<8} {'Nodes':<8}")
    print("-" * 60)
    for result in results:
        cost_str = str(result['cost']) if result['cost'] != float('inf') else "∞"
        print(f"{result['name']:<25} {cost_str:<8} {result['optimal']:<8} {result['nodes_expanded']:<8}")
    
    print(f"\nKEY FINDINGS:")
    print(f"1. Manhattan distance is admissible and finds optimal solution")
    print(f"2. 1.5 * Manhattan is NOT admissible - may find suboptimal paths")
    print(f"3. Inconsistent heuristic may expand more nodes inefficiently")

if __name__ == "__main__":
    run_experiment()

PART C: A* SEARCH IMPLEMENTATION AND ANALYSIS

TESTING: Manhattan Distance
Results:
  Path found: True
  Path: [(0, 0), (0, 1), (0, 2), (0, 3), (1, 3), (2, 3), (2, 4), (2, 5), (3, 5), (3, 6), (3, 7), (4, 7), (4, 8), (4, 9), (5, 9), (6, 9), (7, 9), (8, 9), (9, 9)]
  Cost: 18
  Nodes expanded: 56
  Optimal: True (baseline)

Solution using Manhattan Distance
S * * * # . . . . .
. # # * # . # . . .
. . . * * * # . # .
. # # # # * * * # .
. . . . # . # * * *
# # . . . . . . # *
. . . # # # . . . *
. # . . . . . # # *
. . . # . . . . . *
. . . . . # . . . G


TESTING: 1.5 * Manhattan Distance
Results:
  Path found: True
  Path: [(0, 0), (0, 1), (0, 2), (0, 3), (1, 3), (2, 3), (2, 4), (2, 5), (3, 5), (3, 6), (3, 7), (4, 7), (4, 8), (4, 9), (5, 9), (6, 9), (7, 9), (8, 9), (9, 9)]
  Cost: 18
  Nodes expanded: 19
  Optimal: True

Solution using 1.5 * Manhattan Distance
S * * * # . . . . .
. # # * # . # . . .
. . . * * * # . # .
. # # # # * * * # .
. . . . # . # * * *
# # . . . . . . # *
. . . # 